# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [111]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2f5532311b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [112]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

# Let's filter the properties to see if there's a currency property.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name, ".*currency.*", "i")
}
LIMIT 20
"""

# Greate! we found the currency property IRI P38. Let's look at the objects of the triple
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wdt:P38 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

#We found the euro. We will check its propeties to see what makes it a currency.
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q4916 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

# The Euro is an instance of currency. The BGP of currency should be all the instances of currency IRI Q8142
queryString = """
SELECT DISTINCT ?currency ?name 
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 .
   # get the label
   ?currency sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4591'), ('name', 'Cape Verdean escudo')]
[('currency', 'http://www.wikidata.org/entity/Q17436613'), ('name', 'Java rupiah')]
[('currency', 'http://www.wikidata.org/entity/Q5802556'), ('name', 'dena')]
[('currency', 'http://www.wikidata.org/entity/Q22078645'), ('name', "Chang'an coin")]
[('currency', 'http://www.wikidata.org/entity/Q22907080'), ('name', 'Carthaginian currency')]
[('currency', 'http://www.wikidata.org/entity/Q1542162'), ('name', 'Tarì')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('name', 'Carlino')]
[('currency', 'http://www.wikidata.org/entity/Q383187'), ('name', 'notgeld')]
[('currency', 'http://www.wikidata.org/entity/Q487060'), ('name', 'Denga')]
[('currency', 'http://www.wikidata.org/entity/Q503370'), ('name', 'Kraków złoty')]
[('currency', 'http://www.wikidata.org/entity/Q7567738'), ('name', 'South Korean won')]
[('currency', 'http://www.wikidata.org/entity/Q8047970'), ('name', 'Korean won')]
[(

Final query for this task

In [113]:
# write your final query
queryString = """
SELECT DISTINCT ?currency ?name 
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 .
   # get the label
   ?currency sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q4591'), ('name', 'Cape Verdean escudo')]
[('currency', 'http://www.wikidata.org/entity/Q17436613'), ('name', 'Java rupiah')]
[('currency', 'http://www.wikidata.org/entity/Q5802556'), ('name', 'dena')]
[('currency', 'http://www.wikidata.org/entity/Q22078645'), ('name', "Chang'an coin")]
[('currency', 'http://www.wikidata.org/entity/Q22907080'), ('name', 'Carthaginian currency')]
[('currency', 'http://www.wikidata.org/entity/Q1542162'), ('name', 'Tarì')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('name', 'Carlino')]
[('currency', 'http://www.wikidata.org/entity/Q383187'), ('name', 'notgeld')]
[('currency', 'http://www.wikidata.org/entity/Q487060'), ('name', 'Denga')]
[('currency', 'http://www.wikidata.org/entity/Q503370'), ('name', 'Kraków złoty')]
[('currency', 'http://www.wikidata.org/entity/Q7567738'), ('name', 'South Korean won')]
[('currency', 'http://www.wikidata.org/entity/Q8047970'), ('name', 'Korean won')]
[(

## Task 2
Return the countries which currently use Euro

In [114]:
# write your queries
# When we looked at all the properties of the euro, we found a property "country" IRI P17.
# We can use it to find all the countries that use the euro
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('country', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('country', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('country', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('country', 'http://www.wi

Final query for this task

In [115]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('country', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('country', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('country', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('country', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('country', 'http://www.wi

## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [116]:
# write your queries
# Let's try to find what makes Italy part of the european union
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""
# Let's filter to see if there's a "member" propety
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name, ".*member.*", "i")
}
LIMIT 20
"""

# There actually is a "member of" property IRI P463. I wasn't expecting to find it that easily if I'm being honest.
# Let's see what's on the other side of the predicate and filter for the European Union
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wdt:P463 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER regex(?name, ".*European Union.*", "i")
}
LIMIT 20
"""

# We found the European Union it's IRI is Q458.
# Now let's fetch all European countries and thier currencies
queryString = """
SELECT DISTINCT ?country ?name ?currecny ?currecnyName
WHERE {
   # bind something
   ?country wdt:P463 wd:Q458 ;
            wdt:P38 ?currecny .
   # get the label
   ?country sc:name ?name.
   ?currecny sc:name ?currecnyName.
}
LIMIT 20
"""

# And finaly we execllude countries that use the Euro
queryString = """
SELECT DISTINCT ?country ?name ?currecny ?currecnyName
WHERE {
   # bind something
   {?country wdt:P463 wd:Q458 ;
            wdt:P38 ?currecny .}
   MINUS
   {?country wdt:P463 wd:Q458 ;
            wdt:P38 wd:Q4916 .}
   # get the label
   ?country sc:name ?name.
   ?currecny sc:name ?currecnyName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currecny', 'http://www.wikidata.org/entity/Q123213'), ('currecnyName', 'złoty')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currecny', 'http://www.wikidata.org/entity/Q172540'), ('currecnyName', 'Bulgarian lev')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currecny', 'http://www.wikidata.org/entity/Q47190'), ('currecnyName', 'forint')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currecny', 'http://www.wikidata.org/entity/Q131645'), ('currecnyName', 'Romanian leu')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currecny', 'http://www.wikidata.org/entity/Q25224'), ('currecnyName', 'pound sterling')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currecny', 'http://www.wikidata.org/entity/Q26360'), ('currecnyName', 'Croatian kuna')]
[('country', 'h

Final query for this task

In [117]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?name ?currecny ?currecnyName
WHERE {
   # bind something
   {?country wdt:P463 wd:Q458 ;
            wdt:P38 ?currecny .}
   MINUS
   {?country wdt:P463 wd:Q458 ;
            wdt:P38 wd:Q4916 .}
   # get the label
   ?country sc:name ?name.
   ?currecny sc:name ?currecnyName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currecny', 'http://www.wikidata.org/entity/Q123213'), ('currecnyName', 'złoty')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currecny', 'http://www.wikidata.org/entity/Q172540'), ('currecnyName', 'Bulgarian lev')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currecny', 'http://www.wikidata.org/entity/Q47190'), ('currecnyName', 'forint')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('currecny', 'http://www.wikidata.org/entity/Q131645'), ('currecnyName', 'Romanian leu')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('currecny', 'http://www.wikidata.org/entity/Q25224'), ('currecnyName', 'pound sterling')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currecny', 'http://www.wikidata.org/entity/Q26360'), ('currecnyName', 'Croatian kuna')]
[('country', 'h

## Task 4
For each country that now uses Euro return which was the currencies used before

In [118]:
# write your queries
# Let's first get the countries that use the euro
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
}
LIMIT 30
"""

# Now we get all the currencies of each country
queryString = """
SELECT DISTINCT ?country ?name ?currency  ?currencyName
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   ?currency wdt:P31 wd:Q8142 ; 
             wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
   ?currency sc:name ?currencyName.
}
LIMIT 30
"""

# And finally we filter out the euro
queryString = """
SELECT DISTINCT ?country ?name ?currency  ?currencyName
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   ?currency wdt:P31 wd:Q8142 ; 
             wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
   ?currency sc:name ?currencyName.
   FILTER (str(?currencyName) != "euro")
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2356178'), ('currencyName', 'French Polynesian franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q214393'), ('currencyName', 'CFP Franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2616592'), ('currencyName', 'French West African franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2356117'), ('currencyName', 'New Caledonian franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('currency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName',

Final query for this task

In [119]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?name ?currency  ?currencyName
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?country .
   ?currency wdt:P31 wd:Q8142 ; 
             wdt:P17 ?country .
   # get the label
   ?country sc:name ?name.
   ?currency sc:name ?currencyName.
   FILTER (str(?currencyName) != "euro")
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2356178'), ('currencyName', 'French Polynesian franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q214393'), ('currencyName', 'CFP Franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2616592'), ('currencyName', 'French West African franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q2356117'), ('currencyName', 'New Caledonian franc')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('currency', 'http://www.wikidata.org/entity/Q16068'), ('currencyName',

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [120]:
# write your queries
# That's easy, we just filter for currencies with "dollar" in the name
queryString = """
SELECT DISTINCT ?currency ?name 
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 .
   # get the label
   ?currency sc:name ?name.
   FILTER regex(?name, ".*dollar.*", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('currency', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('currency', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q261447'), ('name', 'Straits dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('currency', 'http://www.wik

Final query for this task

In [121]:
# write your final query
queryString = """
SELECT DISTINCT ?currency ?name 
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 .
   # get the label
   ?currency sc:name ?name.
   FILTER regex(?name, ".*dollar.*", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q3712659'), ('name', 'Texas Dollar')]
[('currency', 'http://www.wikidata.org/entity/Q209792'), ('name', 'Jamaican dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1104069'), ('name', 'Canadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('name', 'Australian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4597'), ('name', 'Solomon Islands dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('name', 'United States dollar')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('name', 'Eastern Caribbean dollar')]
[('currency', 'http://www.wikidata.org/entity/Q2411666'), ('name', 'Newfoundland dollar')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('name', 'Liberian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q261447'), ('name', 'Straits dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4592'), ('name', 'Fijian dollar')]
[('currency', 'http://www.wik

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [122]:
# write your queries
# First we find all currencies and their countries
queryString = """
SELECT DISTINCT ?currency ?name ?country ?countryName
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   # get the label
   ?currency sc:name ?name.
   ?country sc:name ?countryName.
}
LIMIT 30
"""

# Now we count the countries of each currency
queryString = """
SELECT DISTINCT ?currency ?name count(?country) as ?countries
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   # get the label
   ?currency sc:name ?name.
}
GROUP BY ?currency ?name
LIMIT 30
"""

# Lastly we remove currencies used in less than two countries
queryString = """
SELECT DISTINCT ?currency ?name count(?country) as ?countries
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   # get the label
   ?currency sc:name ?name.
}
GROUP BY ?currency ?name
HAVING (COUNT(?country) >= 2)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q16068'), ('name', 'Deutsche Mark'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1852872'), ('name', 'New Hebrides franc'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4124886'), ('name', 'Red Army won'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('name', 'Carlino'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('countries', '23')]
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('name', 'French franc'), ('countries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q618653'), ('name', 'North Yemeni rial'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q106615825'), ('name', 'valadia'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('name', 'Abkhazian apsar'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q373429'), ('name

Final query for this task

In [123]:
# write your final query
queryString = """
SELECT DISTINCT ?currency ?name count(?country) as ?countries
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   # get the label
   ?currency sc:name ?name.
}
GROUP BY ?currency ?name
HAVING (COUNT(?country) >= 2)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q16068'), ('name', 'Deutsche Mark'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1852872'), ('name', 'New Hebrides franc'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4124886'), ('name', 'Red Army won'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q3658911'), ('name', 'Carlino'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro'), ('countries', '23')]
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('name', 'French franc'), ('countries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q618653'), ('name', 'North Yemeni rial'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q106615825'), ('name', 'valadia'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('name', 'Abkhazian apsar'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q373429'), ('name

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [124]:
# write your queries
# Let's fetch the populations of the countries of the last task. We will use only the euro for now to validate the results
queryString = """
SELECT DISTINCT ?currency ?name ?country  ?countryName ?population
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   ?country wdt:P1082 ?population .
   # get the label
   ?currency sc:name ?name.
   ?country sc:name ?countryName.
   FILTER regex(?name, "euro", "i")
}
LIMIT 30
"""

# Now we can group by the currency and su the populations of all the countries
queryString = """
SELECT DISTINCT ?currency ?name sum(?population) as ?population
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   ?country wdt:P1082 ?population .
   # get the label
   ?currency sc:name ?name.
   FILTER regex(?name, "euro", "i")
}
GROUP BY ?currency ?name 
LIMIT 30
"""

# Now we remove the filter for euro
queryString = """
SELECT DISTINCT ?currency ?name sum(?population) as ?population
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   ?country wdt:P1082 ?population .
   # get the label
   ?currency sc:name ?name.
}
GROUP BY ?currency ?name 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q201799'), ('name', 'ngultrum'), ('population', '807610')]
[('currency', 'http://www.wikidata.org/entity/Q186794'), ('name', 'Botswana pula'), ('population', '2291661')]
[('currency', 'http://www.wikidata.org/entity/Q926028'), ('name', 'Brazilian cruzado'), ('population', '213317639')]
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('name', 'Abkhazian apsar'), ('population', '3959100')]
[('currency', 'http://www.wikidata.org/entity/Q4154909'), ('name', 'Danzig mark'), ('population', '366730')]
[('currency', 'http://www.wikidata.org/entity/Q3053201'), ('name', 'Colombian real'), ('population', '49065615')]
[('currency', 'http://www.wikidata.org/entity/Q240292'), ('name', 'South Vietnamese đồng'), ('population', '19582000')]
[('currency', 'http://www.wikidata.org/entity/Q214393'), ('name', 'CFP Franc'), ('population', '67063703')]
[('currency', 'http://www.wikidata.org/entity/Q855244'), ('name', 'Biafran pound'), ('populati

Final query for this task

In [125]:
# write your final query
queryString = """
SELECT DISTINCT ?currency ?name sum(?population) as ?population
WHERE {
   # bind something
   ?currency wdt:P31 wd:Q8142 ;
            wdt:P17 ?country .
   ?country wdt:P1082 ?population .
   # get the label
   ?currency sc:name ?name.
}
GROUP BY ?currency ?name 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q201799'), ('name', 'ngultrum'), ('population', '807610')]
[('currency', 'http://www.wikidata.org/entity/Q186794'), ('name', 'Botswana pula'), ('population', '2291661')]
[('currency', 'http://www.wikidata.org/entity/Q926028'), ('name', 'Brazilian cruzado'), ('population', '213317639')]
[('currency', 'http://www.wikidata.org/entity/Q1983857'), ('name', 'Abkhazian apsar'), ('population', '3959100')]
[('currency', 'http://www.wikidata.org/entity/Q4154909'), ('name', 'Danzig mark'), ('population', '366730')]
[('currency', 'http://www.wikidata.org/entity/Q3053201'), ('name', 'Colombian real'), ('population', '49065615')]
[('currency', 'http://www.wikidata.org/entity/Q240292'), ('name', 'South Vietnamese đồng'), ('population', '19582000')]
[('currency', 'http://www.wikidata.org/entity/Q214393'), ('name', 'CFP Franc'), ('population', '67063703')]
[('currency', 'http://www.wikidata.org/entity/Q855244'), ('name', 'Biafran pound'), ('populati